In [15]:
import pandas as pd
import numpy as np
import altair as alt
import warnings
warnings.filterwarnings('ignore')

In [2]:
print("Starting Focused Analysis: Infrastructure vs. Enrollment/Retention")

Starting Focused Analysis: Infrastructure vs. Enrollment/Retention


In [3]:
print("Phase 1: Loading and Consolidating Data...")

    # Define the correct file paths
path_fac = "/kaggle/input/uidse1/100_fac.csv"
path_prof1 = "/kaggle/input/uidse1/100_prof1.csv"
path_prof2 = "/kaggle/input/uidse1/100_prof2.csv"
path_enr1 = "/kaggle/input/enr-ds/100_enr1.csv"
path_enr2 = "/kaggle/input/enr-ds/100_enr2.csv"

Phase 1: Loading and Consolidating Data...


In [4]:
df_fac = pd.read_csv(path_fac)
df_prof1 = pd.read_csv(path_prof1)
df_prof2 = pd.read_csv(path_prof2)
df_enr1 = pd.read_csv(path_enr1)
df_enr2 = pd.read_csv(path_enr2)

In [5]:
print("All 5 required files loaded successfully.")

All 5 required files loaded successfully.


In [6]:
df_enr = pd.concat([df_enr1, df_enr2], ignore_index=True)
df_prof = pd.merge(df_prof1, df_prof2, on='pseudocode', how='outer')

In [7]:
master_df = df_fac.copy()

In [8]:
master_df = pd.merge(master_df, df_prof, on='pseudocode', how='left')
print(f"Master DataFrame created with {master_df.shape[0]} rows.")
print("Phase 1 Complete.\n")

Master DataFrame created with 1471473 rows.
Phase 1 Complete.



In [9]:
print("Phase 2: Creating Metrics...")

Phase 2: Creating Metrics...


In [10]:
# --- Metric 1: Total Enrollment ---
    
    # 1. Define all enrollment columns
enrollment_cols = ['cpp_b', 'cpp_g', 'c1_b', 'c1_g', 'c2_b', 'c2_g', 
                       'c3_b', 'c3_g', 'c4_b', 'c4_g', 'c5_b', 'c5_g', 
                       'c6_b', 'c6_g', 'c7_b', 'c7_g', 'c8_b', 'c8_g', 
                       'c9_b', 'c9_g', 'c10_b', 'c10_g', 'c11_b', 'c11_g', 
                       'c12_b', 'c12_g']
    
    # Ensure all enrollment columns are numeric, coercing errors
for col in enrollment_cols:
    if col in df_enr.columns:
        df_enr[col] = pd.to_numeric(df_enr[col], errors='coerce')

In [11]:
# 2. Handle missing values (fill with 0)
df_enr[enrollment_cols] = df_enr[enrollment_cols].fillna(0)

In [12]:
# 3. Sum enrollment across all grades for each row
df_enr['school_total_enrollment'] = df_enr[enrollment_cols].sum(axis=1)
# 4. Aggregate by school to get ONE final total per school
df_enr_agg = df_enr.groupby('pseudocode')['school_total_enrollment'].sum().reset_index()
# 5. Merge this total into our Master DataFrame 
master_df = pd.merge(master_df, df_enr_agg, on='pseudocode', how='left')
print("Metric 1 (Total Enrollment) created.")

Metric 1 (Total Enrollment) created.


In [13]:
# --- Metric 2: Infrastructure Score (7-Point Core Needs) ---

# 1. Metric: Building Condition
master_df['metric_building'] = (master_df['pucca_building_blocks'].fillna(0) > 0).astype(int)

# 2. Metric: Functional Toilets
master_df['metric_toilets'] = ((master_df['total_boys_func_toilet'].fillna(0) > 0) & 
                               (master_df['total_girls_func_toilet'].fillna(0) > 0)).astype(int)

# 3. Metric: Drinking Water (Assumes 1=Yes, 0/NaN=No)
master_df['metric_water'] = (master_df['tap_fun_yn'].fillna(0) == 1).astype(int)

# 4. Metric: Electricity (Assumes 1=Yes, 0/NaN=No)
master_df['metric_electricity'] = (master_df['electricity_availability'].fillna(0) == 1).astype(int)

# 5. Metric: Boundary Wall (Assumes 1=Yes, 0/NaN=No)
master_df['metric_boundary'] = (master_df['boundary_wall'].fillna(0) == 1).astype(int)

# 6. Metric: Playground (Assumes 1=Yes, 0/NaN=No)
master_df['metric_playground'] = (master_df['playground_available'].fillna(0) == 1).astype(int)

# 7. Metric: Furniture (Assumes 1=Yes, 0/NaN=No)
master_df['metric_furniture'] = (master_df['furniture_availability'].fillna(0) == 1).astype(int)
# 8. Create the Final Infrastructure Score (out of 7)
score_cols = ['metric_building', 'metric_toilets', 'metric_water', 
              'metric_electricity', 'metric_boundary', 'metric_playground', 'metric_furniture']
master_df['Infrastructure_Score'] = master_df[score_cols].sum(axis=1)
print("Metric 2 (Infrastructure Score) created.")

Metric 2 (Infrastructure Score) created.


In [17]:
# --- Metric 3: Primary Retention Index (Dropout Proxy) ---
    
    # 1. Aggregate C1 and C5 enrollment from the 'df_enr' file
grade_cols = ['c1_b', 'c1_g', 'c5_b', 'c5_g']
for col in grade_cols:
    if col in df_enr.columns:
        df_enr[col] = pd.to_numeric(df_enr[col], errors='coerce').fillna(0)
    else:
        df_enr[col] = 0 # Create empty column to avoid errors
        

In [18]:
df_enr_grades = df_enr.groupby('pseudocode')[grade_cols].sum().reset_index()

In [20]:
# 2. Calculate total C1 and C5 enrollment
df_enr_grades['Total_C1'] = df_enr_grades['c1_b'] + df_enr_grades['c1_g']
df_enr_grades['Total_C5'] = df_enr_grades['c5_b'] + df_enr_grades['c5_g']

In [21]:
# 3. Merge this into the Master DataFrame
master_df = pd.merge(master_df, df_enr_grades[['pseudocode', 'Total_C1', 'Total_C5']], on='pseudocode', how='left')

In [22]:
# 4. Calculate the Retention Index
master_df['highclass'] = pd.to_numeric(master_df['highclass'], errors='coerce') # Ensure highclass is numeric
    
master_df['Primary_Retention_Index'] = np.where(
        (master_df['Total_C1'] > 0) & (master_df['highclass'] == 5),  # Filter for Primary schools with C1 students
        master_df['Total_C5'] / master_df['Total_C1'], 
        np.nan  # Set to NaN for all other schools
    )

In [24]:
# Cap retention at 1.0 (100%)
master_df['Primary_Retention_Index'] = master_df['Primary_Retention_Index'].apply(lambda x: min(x, 1.0) if pd.notnull(x) else np.nan)

In [25]:
print("Metric 3 (Primary Retention Index) created.")
print("Phase 2 Complete.\n")

Metric 3 (Primary Retention Index) created.
Phase 2 Complete.



In [26]:
# ---
    # Phase 3: Focused Analysis
    # ---
print("Phase 3: Performing Focused Analysis...")
    
    # Create the 'Infra_Category' bin for both analyses
master_df['Infra_Category'] = pd.qcut(master_df['Infrastructure_Score'], q=3, labels=['Poor', 'Average', 'Good'], duplicates='drop')

Phase 3: Performing Focused Analysis...


In [28]:
# --- Analysis 1: Infrastructure vs. High Dropout Rates (Retention) ---
    
analysis_1_results = master_df.groupby('Infra_Category')['Primary_Retention_Index'].mean().reset_index()
print("--- Analysis 1: Infrastructure vs. Dropout Proxy (Retention) ---")
print(analysis_1_results.to_markdown(index=False, floatfmt=".2%"))
print("\n")

--- Analysis 1: Infrastructure vs. Dropout Proxy (Retention) ---
| Infra_Category   |   Primary_Retention_Index |
|:-----------------|--------------------------:|
| Poor             |                    80.84% |
| Average          |                    81.76% |
| Good             |                    78.11% |




In [29]:
chart1 = alt.Chart(analysis_1_results).mark_bar().encode(
        x=alt.X('Infra_Category', title='Infrastructure Quality'),
        y=alt.Y('Primary_Retention_Index', title='Average Primary Retention (Class 1-5)', axis=alt.Axis(format='%')),
        tooltip=[
            alt.Tooltip('Infra_Category', title='Infrastructure Quality'),
            alt.Tooltip('Primary_Retention_Index', title='Average Retention', format='.1%')
        ]
    ).properties(
        title='Finding 1: Poor Infrastructure correlates with Higher Dropout Rates'
    )

In [30]:
chart1_path = 'analysis_1_infra_vs_retention.json'
chart1.save(chart1_path)
print(f"Analysis 1 chart saved to {chart1_path}")

Analysis 1 chart saved to analysis_1_infra_vs_retention.json


In [31]:
# --- Analysis 2: Infrastructure vs. Low Student Enrollment ---
    
analysis_2_results = master_df.groupby('Infra_Category')['school_total_enrollment'].mean().reset_index()

print("--- Analysis 2: Infrastructure vs. Total Enrollment ---")
print(analysis_2_results.to_markdown(index=False, floatfmt=",.0f"))
print("\n")

--- Analysis 2: Infrastructure vs. Total Enrollment ---
| Infra_Category   |   school_total_enrollment |
|:-----------------|--------------------------:|
| Poor             |                       309 |
| Average          |                       483 |
| Good             |                       725 |




In [32]:
# --- Plotting for Analysis 2 ---
    
chart2 = alt.Chart(analysis_2_results).mark_bar().encode(
        x=alt.X('Infra_Category', title='Infrastructure Quality'),
        y=alt.Y('school_total_enrollment', title='Average Total Student Enrollment'),
        tooltip=[
            alt.Tooltip('Infra_Category', title='Infrastructure Quality'),
            alt.Tooltip('school_total_enrollment', title='Average Enrollment', format=',.0f')
        ]
    ).properties(
        title='Finding 2: Poor Infrastructure correlates with Lower Student Enrollment'
    )

In [33]:
chart2_path = 'analysis_2_infra_vs_enrollment.json'
chart2.save(chart2_path)
print(f"Analysis 2 chart saved to {chart2_path}")

Analysis 2 chart saved to analysis_2_infra_vs_enrollment.json


In [34]:
# ---
    # Phase 4: Save Output
    # ---
output_file = "school_infra_analysis_master_file.csv"
    
    # Select key columns for the final report
final_columns = [
        'pseudocode',
        'school_category',
        'rural_urban',
        'Infrastructure_Score',
        'Infra_Category',
        'school_total_enrollment',
        'Total_C1',
        'Total_C5',
        'Primary_Retention_Index'
    ]
    
    # Filter for columns that actually exist in master_df
final_columns_exist = [col for col in final_columns if col in master_df.columns]
    
master_df[final_columns_exist].to_csv(output_file, index=False)
print(f"\nAnalysis complete. Full master DataFrame with key metrics saved to:")
print(output_file)


Analysis complete. Full master DataFrame with key metrics saved to:
school_infra_analysis_master_file.csv
